# Secret Santa Matching Algorithms

Hello! You'll be implementing the three different algorithms we discussed two weeks ago and plotting the results:
1. Random
2. Max utility
3. Fairness matching (Your own heuristic!)

## Setup Instructions

**Run this cell first to install required packages:**

In [ ]:
%pip install networkx scipy numpy pandas seaborn matplotlib

## Library imports

In [ ]:
# core libraries for graph-based matching
import networkx as nx
from networkx.algorithms import bipartite
from networkx.algorithms.matching import max_weight_matching

# optimization and numerical computation
from scipy.optimize import linear_sum_assignment
import numpy as np

# randomization
import random

# visualization
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

## Matching Data

In [ ]:
# list of participants
members = ["Charlotte", "Cole", "Joanna", "Justin", "Liam", "Sam", "Samuel", "Stephanie"]

# utility scores for each potential pairing (from receiver's perspective)
# each receiver has preferences for who they would like to receive a gift from
# higher scores = receiver would be happier getting a gift from this person
# note: no self-assignments (no edges from a person to themselves)

# randomly generate preferences between 1-10 for each receiver-giver pair
import random
random.seed(42)  # for reproducability

preferences = {}
for receiver in members:
    preferences[receiver] = {}
    for giver in members:
        if giver != receiver:  # no self-assignments
            preferences[receiver][giver] = random.randint(1, 10)

In [ ]:
# let's look at an example!
print(preferences["Samuel"])

# now let's build a bipartite graph to visualize the matching problem
# givers on the left, receivers on the right
G = nx.Graph()

# add all the givers and receivers as separate nodes
givers = [f"{m}_gives" for m in members]
receivers = [f"{m}_receives" for m in members]
G.add_nodes_from(givers, bipartite=0)
G.add_nodes_from(receivers, bipartite=1)

# add edges between givers and receivers with weights from preferences
# skip self-loops since you can't give yourself a gift!
for receiver in members:
    for giver in members:
        if giver != receiver:
            weight = preferences[receiver][giver]
            G.add_edge(f"{giver}_gives", f"{receiver}_receives", weight=weight)

# set up positions: givers on left (x=0), receivers on right (x=2)
pos = {}
y_positions = np.linspace(0, len(members)-1, len(members))
for i, giver in enumerate(givers):
    pos[giver] = (0, y_positions[i])
for i, receiver in enumerate(receivers):
    pos[receiver] = (2, y_positions[i])

# draw the graph!
plt.figure(figsize=(14, 10))

# draw edges with thickness based on preference weight
for edge in G.edges(data=True):
    giver_node, receiver_node, data = edge
    weight = data['weight']
    # thicker edges mean higher preference (maps 1-10 to line width 0.5-5)
    thickness = 0.5 + (weight - 1) * 0.5
    nx.draw_networkx_edges(G, pos, [(giver_node, receiver_node)], 
                           width=thickness, alpha=0.4, edge_color='gray')

# draw the giver nodes (blue squares on left)
nx.draw_networkx_nodes(G, pos, nodelist=givers, 
                       node_color='#3498DB', node_size=600, node_shape='s')

# draw the receiver nodes (green circles on right)
nx.draw_networkx_nodes(G, pos, nodelist=receivers, 
                       node_color='#2ECC71', node_size=600, node_shape='o')

# add labels (clean up the _gives and _receives suffixes)
labels = {node: node.replace('_gives', '').replace('_receives', '') 
          for node in G.nodes()}
nx.draw_networkx_labels(G, pos, labels, font_size=9)

plt.title('secret santa bipartite graph\\n(thicker edges = higher preference)', 
          fontsize=14, pad=20)
plt.axis('off')
plt.tight_layout()
plt.show()

# TASK 1: RANDOM MATCHING

In [ ]:
def random_matching_statistics(members, preferences):
    """
    calculates the expected statistics for a random perfect matching (baseline algorithm).
    
    for a random perfect matching, each person has equal probability of being
    matched to any other person (excluding themselves). you should calculate the 
    expected mean and std for EACH person, then find the overall expected mean and std.
    
    mathematical approach:
        - for each receiver, their expected utility = average of all their preference values
        - their expected std = standard deviation of all their preference values
        - overall expected mean = average of all individual expected means
        - overall expected std = you'll need to think about how to combine individual stds!
    
    args:
        members (list): list of participant names
        preferences (dict): nested dictionary of utility scores (receiver -> giver -> score)
    
    returns:
        dict: statistics dictionary with mean and std for each person, plus overall mean and std
    
    hints:
        - use np.mean() to calculate mean of a list
        - use np.std() to calculate standard deviation of a list
        - for each person, get all their preference values as a list
        - calculate their individual expected mean and std
    """
    stats_random = {
        "mean": {
            "Charlotte": 0,
            "Cole": 0,
            "Joanna": 0,
            "Justin": 0,
            "Liam": 0,
            "Sam": 0,
            "Samuel": 0,
            "Stephanie": 0
        },
        "standard_deviation": {
            "Charlotte": 0,
            "Cole": 0,
            "Joanna": 0,
            "Justin": 0,
            "Liam": 0,
            "Sam": 0,
            "Samuel": 0,
            "Stephanie": 0
        },
        "overall_mean": 0,
        "overall_std": 0
    }
    
    # TODO: for each member, calculate their expected mean and std
    # hint: for each receiver in members:
    #           get all their preference values: list(preferences[receiver].values())
    #           calculate mean using np.mean()
    #           calculate std using np.std()
    #           store in stats_random["mean"][receiver] and stats_random["standard_deviation"][receiver]
    
    # TODO: calculate overall expected mean
    # hint: average all the individual means you just calculated
    # store it as stats_random["overall_mean"]
    
    # TODO: calculate overall expected std
    # hint: think about how to combine individual standard deviations (it's more complicated than you think, you can also just aggregate the results and do it that way)
    # store it as stats_random["overall_std"]
    
    return stats_random

# TASK 2: MAXIMUM UTILITY MATCHING

In [ ]:
def maximum_utility_matching(members, preferences):
    """
    finds the secret santa assignment that maximizes total group happiness.
    
    this is the classic maximum weight bipartite matching problem. you can use
    either networkx or scipy:
    
    option 1 - networkx (recommended):
        - create a weighted graph with nx.Graph()
        - add edges with weights from preferences
        - use max_weight_matching() function
    
    option 2 - scipy (faster, but requires matrix conversion):
        - convert preferences dict to cost matrix (numpy array)
        - use linear_sum_assignment(cost_matrix, maximize=True)
        - map indices back to member names
    
    args:
        members (list): list of participant names
        preferences (dict): nested dictionary of utility scores
    
    returns:
        tuple: (matching, stats) where matching is {giver: receiver} and stats is the stats dict
    
    hints:
        - first find the optimal matching (giver -> receiver pairs)
        - then for each receiver, look up their utility from the matching
        - calculate mean and std for each person's utility
        - use np.mean() and np.std()
    """
    # your code here
    matching = {}
    
    # TODO: implement maximum utility matching
    # hint: use networkx.max_weight_matching() or scipy.optimize.linear_sum_assignment()
    
    # after you find the matching, calculate stats for each person
    stats_max_utility = {
        "mean": {
            "Charlotte": 0,
            "Cole": 0,
            "Joanna": 0,
            "Justin": 0,
            "Liam": 0,
            "Sam": 0,
            "Samuel": 0,
            "Stephanie": 0
        },
        "standard_deviation": {
            "Charlotte": 0,
            "Cole": 0,
            "Joanna": 0,
            "Justin": 0,
            "Liam": 0,
            "Sam": 0,
            "Samuel": 0,
            "Stephanie": 0
        },
        "overall_mean": 0,
        "overall_std": 0
    }
    
    # TODO: for each member, get their utility from the matching
    # hint: for each receiver, find who gives to them in the matching
    #       then look up preferences[receiver][giver] to get their utility
    #       store it in stats_max_utility["mean"][receiver]
    #       (for a single matching, each person gets one value, so std will be 0)
    
    # TODO: calculate overall mean
    # hint: average all the individual utilities
    # store it as stats_max_utility["overall_mean"]
    
    # TODO: calculate overall std
    # hint: standard deviation of all individual utilities
    # store it as stats_max_utility["overall_std"]
    
    return matching, stats_max_utility

# TASK 3: MAXIMUM FAIRNESS MATCHING

In [ ]:
def maximum_fairness_matching(members, preferences):
    """
    implements a custom fairness algorithm for secret santa assignments.
    
    IMPORTANT: you must define what "fairness" means for your team's algorithm.
    
    possible fairness definitions:
        - minimize variance in utility scores (everyone gets similar happiness)
        - minimax approach (maximize the minimum utility score)
        - balanced distribution (avoid very high and very low pairings)
        - egalitarian matching (ensure no one gets their worst choice)
    
    args:
        members (list): list of participant names
        preferences (dict): nested dictionary of utility scores
    
    returns:
        tuple: (matching, stats) where matching is {giver: receiver} and stats is the stats dict
    
    hints:
        - first define and implement your fairness metric
        - find the matching that best satisfies your fairness criteria
        - then calculate stats for each person from that matching
        - use np.mean() and np.std()
    """
    # YOUR FAIRNESS DEFINITION:
    # TODO: describe your fairness metric here
    
    # your code here
    matching = {}
    
    # TODO: implement your custom fairness algorithm
    
    # after you find the matching, calculate stats for each person
    stats_fairness = {
        "mean": {
            "Charlotte": 0,
            "Cole": 0,
            "Joanna": 0,
            "Justin": 0,
            "Liam": 0,
            "Sam": 0,
            "Samuel": 0,
            "Stephanie": 0
        },
        "standard_deviation": {
            "Charlotte": 0,
            "Cole": 0,
            "Joanna": 0,
            "Justin": 0,
            "Liam": 0,
            "Sam": 0,
            "Samuel": 0,
            "Stephanie": 0
        },
        "overall_mean": 0,
        "overall_std": 0
    }
    
    # TODO: for each member, get their utility from the matching
    # hint: for each receiver, find who gives to them in the matching
    #       then look up preferences[receiver][giver] to get their utility
    #       store it in stats_fairness["mean"][receiver]
    #       (for a single matching, each person gets one value, so std will be 0)
    
    # TODO: calculate overall mean
    # hint: average all the individual utilities
    # store it as stats_fairness["overall_mean"]
    
    # TODO: calculate overall std  
    # hint: standard deviation of all individual utilities
    # store it as stats_fairness["overall_std"]
    
    return matching, stats_fairness

# Visualizations

now that we have all the results stored, let's visualize them!
Note, you don't need to edit the code here

In [ ]:
def plot_mean_comparison(results):
    """Bar chart comparing mean utilities with std dev error bars"""
    algorithms = list(results.keys())
    means = [results[alg]['mean'] for alg in algorithms]
    stds = [results[alg]['std'] for alg in algorithms]
    
    plt.figure(figsize=(10, 6))
    plt.bar(algorithms, means, yerr=stds, capsize=10, alpha=0.7, color=['#FF6B6B', '#4ECDC4', '#45B7D1'])
    plt.xlabel('Algorithm', fontsize=12)
    plt.ylabel('Mean Utility', fontsize=12)
    plt.title('Algorithm Comparison: Mean Utility with Standard Deviation', fontsize=14, fontweight='bold')
    plt.grid(axis='y', alpha=0.3)
    plt.tight_layout()
    plt.show()

def plot_individual_utilities(results, members):
    """Grouped bar chart showing individual member utilities across algorithms"""
    algorithms = list(results.keys())
    fig, ax = plt.subplots(figsize=(14, 6))
    
    x = np.arange(len(members))
    width = 0.25
    
    for i, alg in enumerate(algorithms):
        offset = (i - 1) * width
        utilities = results[alg]['individual_utilities']
        ax.bar(x + offset, utilities, width, label=alg, alpha=0.8)
    
    ax.set_xlabel('Members', fontsize=12)
    ax.set_ylabel('Utility', fontsize=12)
    ax.set_title('Individual Member Utilities by Algorithm', fontsize=14, fontweight='bold')
    ax.set_xticks(x)
    ax.set_xticklabels(members, rotation=45, ha='right')
    ax.legend()
    ax.grid(axis='y', alpha=0.3)
    plt.tight_layout()
    plt.show()

def plot_happiness_distribution(results):
    """Pie charts showing happiness distribution (Happy/OK/Sad)"""
    algorithms = list(results.keys())
    fig, axes = plt.subplots(1, len(algorithms), figsize=(15, 5))
    
    for i, alg in enumerate(algorithms):
        utilities = results[alg]['individual_utilities']
        
        # Categorize utilities: Happy (7-10), OK (4-6), Sad (1-3)
        happy = sum(1 for u in utilities if u >= 7)
        ok = sum(1 for u in utilities if 4 <= u < 7)
        sad = sum(1 for u in utilities if u < 4)
        
        # Create pie chart
        sizes = [happy, ok, sad]
        labels = ['Happy (7-10)', 'OK (4-6)', 'Sad (1-3)']
        colors = ['#2ECC71', '#F39C12', '#E74C3C']
        explode = (0.05, 0, 0)
        
        axes[i].pie(sizes, explode=explode, labels=labels, colors=colors, autopct='%1.1f%%',
                    shadow=True, startangle=90)
        axes[i].set_title(f'{alg}\nHappiness Distribution', fontweight='bold')
    
    plt.tight_layout()
    plt.show()

# Main execution

Run all algorithms and store results for visualization

In [ ]:
# initialize results storage
results = {}

print("SECRET SANTA MATCHING ALGORITHM COMPARISON")
print("P-resents Simulation Engine - Prototype v1.0")
print("="*60)

## test task 1: random matching

In [ ]:
# task 1: random matching (expected statistics)
print("\nTASK 1: RANDOM MATCHING (Expected Statistics)")
print("-"*60)
random_stats = random_matching_statistics(members, preferences)
print(f"Expected Mean Utility: {random_stats['overall_mean']:.2f}")
print(f"Expected Std Dev: {random_stats['overall_std']:.2f}")

# store random matching results
# convert individual means dict to list in the same order as members
individual_means_list = [random_stats['mean'][member] for member in members]

results['Random'] = {
    'mean': random_stats['overall_mean'],
    'std': random_stats['overall_std'],
    'individual_utilities': individual_means_list
}

print("\n✓ task 1 complete! results stored.")

## test task 2: maximum utility matching

In [ ]:
# task 2: maximum utility matching
print("\nTASK 2: MAXIMUM UTILITY MATCHING")
print("-"*60)
utility_match, utility_stats = maximum_utility_matching(members, preferences)
print(f"Total Utility: {utility_stats['overall_mean'] * len(members):.0f}")
print(f"Mean Utility: {utility_stats['overall_mean']:.2f}")
print(f"Std Dev: {utility_stats['overall_std']:.2f}")

# convert individual means dict to list
individual_utilities_list = [utility_stats['mean'][member] for member in members]

# store max utility results
results['Maximum Utility'] = {
    'mean': utility_stats['overall_mean'],
    'std': utility_stats['overall_std'],
    'individual_utilities': individual_utilities_list
}

print("\n✓ task 2 complete! results stored.")

## test task 3: maximum fairness matching

In [ ]:
# task 3: maximum fairness matching
print("\nTASK 3: MAXIMUM FAIRNESS MATCHING")
print("-"*60)
fairness_match, fairness_stats = maximum_fairness_matching(members, preferences)
print(f"Total Utility: {fairness_stats['overall_mean'] * len(members):.0f}")
print(f"Mean Utility: {fairness_stats['overall_mean']:.2f}")
print(f"Std Dev: {fairness_stats['overall_std']:.2f}")

# convert individual means dict to list
individual_fairness_list = [fairness_stats['mean'][member] for member in members]

# store fairness results
results['Maximum Fairness'] = {
    'mean': fairness_stats['overall_mean'],
    'std': fairness_stats['overall_std'],
    'individual_utilities': individual_fairness_list
}

print("\n✓ task 3 complete! results stored.")

## comparison summary

In [ ]:
# summary table
print(f"\n{'='*60}")
print("COMPARISON SUMMARY")
print(f"{'='*60}")
print(f"{'Algorithm':<30s} {'Mean Utility':>15s} {'Std Dev':>12s}")
print("-" * 60)

for name in results.keys():
    print(f"{name:<30s} {results[name]['mean']:>15.2f} {results[name]['std']:>12.2f}")